In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/artificial_vision')

Mounted at /content/drive


In [2]:
!unzip training_set.zip
!unzip validation_set.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: training_set/CAM28-2014-02-21-20140221121939-20140221123218-tarid127-frame6398-line1.jpg  
  inflating: training_set/VideoX_321_50_1.jpg  
  inflating: training_set/CAM12-2014-03-10-20140310130107-20140310130655-tarid585-frame3352-line1.jpg  
  inflating: training_set/CAM18-2014-04-15-20140415111709-20140415112257-tarid192-frame3108-line1.jpg  
  inflating: training_set/VideoX_303_438_3.jpg  
  inflating: training_set/CAM22-2014-04-23-20140423164934-20140423165602-tarid0-frame313-line2.jpg  
  inflating: training_set/CAM01-2014-03-20-20140320163814-20140320165406-tarid178-frame10944-line1.jpg  
  inflating: training_set/VideoX_309_371_1.jpg  
  inflating: training_set/CAM17-2013-12-23-20131223121202-20131223121750-tarid16-frame589-line1.jpg  
  inflating: training_set/CAM25-2014-03-25-20140325115406-20140325115950-tarid436-frame4042-line3.jpg  
  inflating: training_set/CAM27-2014-03-21-20140321150841-20140321151441-tarid15

In [2]:
import torch
from torch.utils.data import Sampler
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torchvision.models import resnet50
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from torch.utils.data import RandomSampler
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
  print("You have %d GPUs" % torch.cuda.device_count())
  gpu_id = torch.cuda.current_device()
  print("The selected GPU is GPU", gpu_id)
  print("- Name:", torch.cuda.get_device_name(gpu_id))
  print("- All properties:",torch.cuda.get_device_properties(gpu_id))

You have 1 GPUs
The selected GPU is GPU 0
- Name: Tesla T4
- All properties: _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [3]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

def extract_annotation(label_path):
  annotations_list = []

  with open(label_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        elements = line.split(',')
        # Creazione di una tupla (puoi usare una lista se preferisci)
        annotation_tuple = (
            elements[0],  # Nome dell'immagine
            int(elements[1]),  # Label dell'abbigliamento superiore (convertita in intero)
            int(elements[2]),  # Label dell'abbigliamento inferiore
            int(elements[3]),  # Label del genere della persona
            int(elements[4]),  # Label relativa alla presenza del cappello
            int(elements[5])   # Label relativa alla presenza della borsa
        )
        # Aggiungi la tupla alla lista delle annotazioni
        annotations_list.append(annotation_tuple)

  annotations_list.sort(key=lambda x: x[0])
  return annotations_list


training_folder = 'training_set'
validation_folder = 'validation_set'

train_label = 'training_set.txt'
validation_label = 'validation_set.txt'

annotations_training_list = extract_annotation(train_label)
annotations_validation_list = extract_annotation(validation_label)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.RandomAffine(degrees=30, shear=10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        normalize
    ]),
}


training_image_paths = [os.path.join(training_folder, filename) for filename in os.listdir(training_folder)]
validation_image_paths = [os.path.join(validation_folder, filename) for filename in os.listdir(validation_folder)]


training_image_paths.sort()
validation_image_paths.sort()

print(len(training_image_paths))
print(len(validation_image_paths))

4574
4574


In [4]:
images = os.listdir('./training_set')

print(len(images))

# with open('training_set.txt', 'r') as file:
#     # Legge tutte le righe del file in una lista
#     lines = file.readlines()
#     print(len(lines))
#     # for line in lines:
#     #   line_splitted = line.split(',')[0]

#     #   if line_splitted not in images:
#     #     print(line_splitted)
# # Calcola il numero di righe nel file





4605


In [ ]:
from collections import defaultdict

class CustomImageDataset(Dataset):

    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths # lista di path
        self.labels = labels    # lista di tuple (nome, att1, att2, att3, att4, att5)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)


    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]# label è una tupla
        label = torch.tensor(list(label[1:]), dtype=torch.float)

        image = Image.open(image_path).convert('RGB')

        if self.transform:  # passato all esterno
            image = self.transform(image)

        return image, label

    @staticmethod
    def make_color_weights(dataset):
        upper_dict = defaultdict(int)  # key: label da 1 a 11, value: numero di occorrenze
        lower_dict = defaultdict(int)

        for label in dataset.labels:
            upper = label[1]
            lower = label[2]

            if upper != -1:
                upper_dict[upper] += 1

            if lower != -1:
                lower_dict[lower] += 1

        key_upper = list(upper_dict.keys())
        key_lower = list(lower_dict.keys())

        key_upper = sorted(key_upper)
        key_lower = sorted(key_lower)


        upper_weights = torch.tensor([1 / upper_dict[key] for key in key_upper])
        lower_weights = torch.tensor([1 / lower_dict[key] for key in key_lower])

        return upper_weights, lower_weights


class CustomRandomSampler(Sampler):

    def __init__(self, dataset, replacement=False, batch_size=None):
        self.data_source = dataset
        self.replacement = replacement
        self.batch_size = batch_size

    def __iter__(self):
        indices = list(range(len(self.data_source)))

        number_iteration = len(indices) // self.batch_size

        if not self.replacement:
            indices = np.random.permutation(indices).tolist()

        batch = []
        index_seen = set()
        deque = []
        iteration = 1

        for idx in indices:
            if len(deque) > 0:
                t = min(self.batch_size - len(batch), len(deque))
                batch.extend(deque[:t])
                del deque[:t]
                if len(batch) == self.batch_size:
                    deque.append(idx)

            if len(batch) < self.batch_size:
                batch.append(idx)

            if len(batch) == self.batch_size:

                for batch_sample_index, sample_index in enumerate(batch):
                    _, labels = self.data_source[sample_index]

                    for label_index, l in enumerate(labels):

                        if l != -1:
                            index_seen.add(label_index)
                        elif l == -1 and label_index not in index_seen:

                            deque.append(sample_index)

                            # tentative = 0
                            while(True):
                                index_new_sample = np.random.choice(len(self.data_source))
                                _, labels = self.data_source[index_new_sample]
                                # tentative += 1
                                if labels[label_index] != -1:
                                    batch[batch_sample_index] = index_new_sample
                                    break

                yield batch

                if iteration == number_iteration:
                  break

                iteration += 1
                index_seen = set()
                batch = []


training_dataset = CustomImageDataset(training_image_paths, annotations_training_list, transform=data_transforms['train'])
validation_dataset = CustomImageDataset(validation_image_paths, annotations_validation_list, transform=data_transforms['validation'])

batch_size = 32
sampler = CustomRandomSampler(dataset=training_dataset, batch_size=batch_size, replacement=False)

train_dataloader = DataLoader(training_dataset, batch_sampler = sampler)
val_dataloader = DataLoader(validation_dataset, batch_size = batch_size, shuffle = False)

NameError: name 'CustomImageDataset' is not defined

In [ ]:
for b in train_dataloader:
  x, labels = b
  print('-----')
  print(x)
  print('----')
  print(labels)

-----
tensor([[[[ 1.1529,  1.1358,  1.1187,  ..., -2.1179, -2.1179, -2.1179],
          [ 1.1529,  1.1358,  1.1358,  ..., -2.1179, -2.1179, -2.1179],
          [ 1.1700,  1.1529,  1.1358,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -0.3541, -0.3198, -0.2513],
          [-2.1179, -2.1179, -2.1179,  ..., -0.2856, -0.2856, -0.1999],
          [-2.1179, -2.1179, -2.1179,  ..., -0.3369, -0.2342, -0.1828]],

         [[ 1.2206,  1.2031,  1.2031,  ..., -2.0357, -2.0357, -2.0357],
          [ 1.2206,  1.2031,  1.2031,  ..., -2.0357, -2.0357, -2.0357],
          [ 1.2381,  1.2206,  1.2031,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -0.4601, -0.4251, -0.3550],
          [-2.0357, -2.0357, -2.0357,  ..., -0.4076, -0.4076, -0.3025],
          [-2.0357, -2.0357, -2.0357,  ..., -0.4601, -0.3550, -0.2850]],

         [[ 1.3851,  1.3677,  1.3677,  ..., -1.8044, -1.8044, -1.8044],
          [ 1.3851,  1.3

KeyboardInterrupt: 

In [ ]:
dataloader = train_dataloader
index=4

# Display image and label.
val_features, val_labels = next(iter(dataloader))
print(f"Feature batch shape: {len(val_features)}")
img = val_features[index].squeeze()
label = val_labels[index]
plt.imshow(img.permute(1,2,0), cmap="gray")
plt.show()
print(f"Label: {val_labels}")

In [ ]:

class ResNet50Backbone(nn.Module):

    def __init__(self):
        super(ResNet50Backbone, self).__init__() # Chiama il costruttore della classe madre nn.Module per inizializzare la classe base.

        self.model = resnet50(pretrained=True) # Carica il modello preaddestrato ResNet-50 dal torchvision.models con i pesi preaddestrati su ImageNet.
        self.model = torch.nn.Sequential(*list(self.model.children())[:-1])

    def forward(self, x):
        return self.model(x)

    def freeze_all(self):
        for param in self.model.parameters():
            param.requires_grad = False


In [ ]:
class AttentionModule(nn.Module):

    def __init__(self, in_channels):
        super(AttentionModule, self).__init__()

        self.channel_attention  = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // 16, kernel_size=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 16, in_channels, kernel_size=1, padding=0),
            nn.Sigmoid()
        )

        self.spatial_attention = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        channel_att = self.channel_attention(x)
        print("channel_attention: ", channel_att.size())
        spatial_att = self.spatial_attention(x)
        print("spatial_attention: ", spatial_att.size())
        x = x * channel_att * spatial_att
        return x

In [ ]:
class ClassifierBlock(nn.Module):

  def __init__(self, layer, activation_function):
     super(ClassifierBlock, self).__init__()

     self.block = nn.Sequential(layer, activation_function)

  def forward(self, x):
    return self.block(x)

In [ ]:
class AttributeRecognitionModel(nn.Module):

    def __init__(self, num_attributes):
        super(AttributeRecognitionModel, self).__init__()

        # Backbone ResNet-50
        self.backbone = ResNet50Backbone()

        # Moduli di attenzione spaziale e di canale per ogni attributo
        self.attention_modules = nn.ModuleList([AttentionModule(in_channels=2048) for _ in range(num_attributes)]) # Crea una lista di moduli di attenzione spaziale e di canale per ogni attributo.
        # nn.ModuleList viene utilizzato per contenere i moduli in modo che PyTorch li tracci correttamente come parte del modello.

        # Classificatori per ogni attributo
        binary_classifier = [ClassifierBlock(nn.Linear(2048, 2), nn.Sigmoid()) for _ in range(3)]

        multi_classifier = [ClassifierBlock(nn.Linear(2048, 11), nn.Softmax(dim=1)) for _ in range(2)]

        self.classifiers = nn.ModuleList(multi_classifier + binary_classifier) # Crea una lista di classificatori lineari per ogni attributo.

    def forward(self, x):
        # Passa l'input attraverso il backbone
        features = self.backbone(x)
        print("dimensione di features: ", features.size())
        pred_list=[]
        attention_outputs = [attention(features) for attention in self.attention_modules]

        for att_output, classifier in zip(attention_outputs, self.classifiers):
            print("Dimensione att_output:", att_output.size())
            flattened_output = att_output.view(att_output.size(0), -1)
            print("Dimensione flattened_output:", flattened_output.size())
            pred = classifier(flattened_output)
            print("Dimensione logits:", pred.size())
            pred_list.append(pred)
        print("Dimensione logits totale:", len(pred_list))
        print('--- pred_list ', pred_list)
        return pred_list

    def freeze_backbone_parameters(self):
      self.backbone.freeze_all()

    def unfreeze_parameters(self):
        for param in self.attention_modules.parameters():
            param.requires_grad = True

        for param in self.classifiers.parameters():
            param.requires_grad = True




In [ ]:
class BinaryAsymmetricLoss(nn.Module):
  # https://arxiv.org/abs/2009.14119

  def __init__(self, gamma_neg: any, gamma_pos: any, eps: float = 1e-8, ignore_index = None) -> None:
    super(BinaryAsymmetricLoss, self).__init__()
    self.gamma_neg = torch.tensor(gamma_neg)
    self.gamma_pos = torch.tensor(gamma_pos)
    self.eps = eps
    self.ignore_index = ignore_index

  def forward(self, prob: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    targets = targets.view(-1, 1)
    term_neg = (1 - targets) * (prob ** self.gamma_neg) * torch.log(1 - prob + self.eps)
    term_pos = (targets * (1 - prob) ** self.gamma_pos) * torch.log(prob + self.eps)

    if self.ignore_index:
        mask = torch.ones_like(targets, dtype=torch.bool)
        mask[targets == self.ignore_index] = 0
        mask = mask.expand_as(term_neg)
        term_neg = term_neg[mask]
        term_pos = term_pos[mask]

    return -torch.mean(term_neg + term_pos)

# prob = torch.tensor([[0.7, 0.3], [0.4, 0.6], [0.2, 0.8]])
# targets = torch.tensor([0, 1, -1])  # assuming -1 for ignored index
# print(targets)
# gamma_neg = 1.0
# gamma_pos = 1.0

# binary_criterion = BinaryAsymmetricLoss(gamma_neg, gamma_pos, ignore_index=-1)
# loss = binary_criterion(prob, targets)
# print(loss)


tensor([ 0,  1, -1])
tensor(0.4260)


In [ ]:
import torch
class Accuracy():
    def __init__(self, threshold=0.5, is_binary=False):
        self.threshold = threshold
        self.is_binary = is_binary

    def __call__(self, predictions, targets):
        prob_max, prob_indicies = torch.max(predictions, dim=1)

        if self.is_binary:
            cond1 = prob_max > self.threshold
            cond2 = prob_indicies == targets.detach()
            acc = cond1 * cond2
            acc = acc.float().mean()
            return acc

        prob_indicies = prob_indicies + 1
        cond = prob_indicies == targets.detach()
        acc = cond.float().mean()
        return acc

# binary_acc = Accuracy(threshold=0.5, is_binary=True)
# multi_acc = Accuracy(threshold=0.5, is_binary=False)

# t1_p = torch.tensor([[0.1, 0.9], [0.3, 0.7], [0.3, 0.7]])
# t1_t = torch.tensor([1, 0, 1])
# print(binary_acc(t1_p, t1_t))
# t2_p = torch.tensor([[0.05, 0.0, 0.2, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [0.05, 0.0, 0.15, 0.15, 0.05, 0, 0.15, 0.15, 0.1, 0.1, 0.1]])
# t2_t = torch.tensor([3, 2])
# print(multi_acc(t2_p, t2_t))
# # print(binary_acc(t1_p, t1_t))

tensor([1., 0., 1.])


In [ ]:
num_attributes = 5

model = AttributeRecognitionModel(num_attributes=num_attributes)
model.freeze_backbone_parameters()
model.unfreeze_parameters()

upper_weights, lower_weights = CustomImageDataset.make_color_weights(training_dataset)

loss_list = [nn.CrossEntropyLoss(weight=upper_weights, ignore_index=-1), nn.CrossEntropyLoss(weight=lower_weights, ignore_index=-1), BinaryAsymmetricLoss(ignore_index=-1, gamma_neg=1.0, gamma_pos=1.0), BinaryAsymmetricLoss(ignore_index=-1, gamma_neg=1.0, gamma_pos=1.0), BinaryAsymmetricLoss(ignore_index=-1, gamma_neg=1.0, gamma_pos=1.0)]
accuracy_list = [Accuracy(is_binary=False), Accuracy(is_binary=False), Accuracy(is_binary=True), Accuracy(is_binary=True), Accuracy(is_binary=True)]
optimizer = optim.Adam(model.parameters(), lr=1e-2)



In [ ]:
import tqdm

def one_epoch(model, criterion_list, optimizer, train_loader, val_loader, epoch_num, accuracy_list):
  model.cuda()
  model.train()

  for i, (images, labels) in tqdm(enumerate(train_loader), desc="epoch {} - train".format(epoch_num)):

    images = images.cuda()
    labels = labels.cuda().float()
    optimizer.zero_grad()

    o = model(images)

    result_losses = []
    result_accuracies = []
    for attr_index in range(num_attributes):
      target = labels[:, attr_index]
      # print("dim di target_labels: ", target_labels.size())
      # print('----- target labels ', target)
      attribute_predictions = o[attr_index]
      # print('---- attribute predictions', attribute_predictions)
      # print("dim di attribute_predictions: ", attribute_predictions.size())
      # predicted_probabilities, _ = torch.max(attribute_predictions, dim=1)
      loss = criterion_list[attr_index](attribute_predictions, target)
      acc = accuracy_list[attr_index](attribute_predictions, target)

      result_accuracies.append(acc)
      result_losses.append(loss)

    total_loss = sum(result_losses)
    total_loss.backward()
    optimizer.step()

    # print(f'- batch loss: {total_loss.item():.7f} {sum(result_accuracies)/len(result_accuracies):.4f}')

  model.eval()
  with torch.no_grad():
    val_loss = []
    val_acc = []

    for images, labels in tqdm(val_loader, desc="epoch {} - validation".format(epoch_num)):
      images = images.cuda()
      labels = labels.cuda().float()

      o = model(images)

      batch_loss = []
      batch_acc = []
      for attr_index in range(num_attributes):
        target = labels[:, attr_index]
        attribute_predictions = o[attr_index]

        loss = criterion_list[attr_index](attribute_predictions, target)
        acc = accuracy_list[attr_index](attribute_predictions, target)

        batch_loss.append(loss)
        batch_acc.append(acc.item())

      aggregated_loss = sum(batch_loss)
      val_loss.append(aggregated_loss)

      val_acc.extend(batch_acc)



      val_acc.append(acc)

    val_loss = torch.stack(val_loss).mean().item()
    val_accuracy = sum(val_acc) / num_attributes

    print("Validation loss and accuracy : {:.7f}\t{:.4f}".format(val_loss, val_accuracy))

  return val_loss, torch.tensor(val_accuracy)


# early stopping and best model saving
EARLY_STOPPIMG_PATIENCE = 5
early_stopping_counter = EARLY_STOPPIMG_PATIENCE

epochs = 1
min_val_loss = 1e10

# training and validation
val_losses = torch.zeros(epochs)
val_accuracies = torch.zeros(epochs)

for e in range(epochs):
  print("EPOCH {}".format(e))

  val_loss, val_accuracy = one_epoch(model, loss_list, optimizer, train_dataloader, val_dataloader, e, accuracy_list)

  # store the validation metrics
  val_losses[e] = val_loss
  val_accuracies[e] = val_accuracy

  # save the best model and check the early stopping criteria
  if val_loss < min_val_loss: # save the best model
    min_val_loss = val_loss
    early_stopping_counter = EARLY_STOPPIMG_PATIENCE # reset early stopping counter
    torch.save(model.state_dict(), os.path.join('par_models','best_model.pth'))
    print("- saved best model: val_loss =", val_loss, "val_accuracy =", val_accuracy)

  if e>0: # early stopping counter update
    if val_losses[e] > val_losses[e-1]:
        early_stopping_counter -= 1 # update early stopping counter
    else:
        early_stopping_counter = EARLY_STOPPIMG_PATIENCE # reset early stopping counter

  if early_stopping_counter == 0: # early stopping
      break

In [ ]:
# VECCHIO CODICE

num_epochs = 1

# Addestramento del modello
for epoch in range(num_epochs):
    model.train()

    for images, labels in val_dataloader:
        # print("dim di images: ", images.size())
        # print("dim di labels: ", labels.size())
        # print('----', labels)
        optimizer.zero_grad()
        outputs = model(images)
        losses = []
        for attr_index in range(num_attributes):
            target = labels[:, attr_index]
            # print("dim di target_labels: ", target_labels.size())
            print('----- target labels ', target)
            attribute_predictions = outputs[attr_index]
            print('---- attribute predictions', attribute_predictions)
            # print("dim di attribute_predictions: ", attribute_predictions.size())
            predicted_probabilities, _ = torch.max(attribute_predictions, dim=1)
            loss = criterion(predicted_probabilities, target)
            losses.append(loss)
        total_loss = sum(losses)
        total_loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss.item()}')